In [1]:
from cerebro.backend import SparkBackend
from cerebro.keras import SparkEstimator

# datas storage for intermediate data and model artifacts.
from cerebro.storage import LocalStore, HDFSStore

# Model selection/AutoML methods.
from cerebro.tune import GridSearch, RandomSearch, TPESearch

# Utility functions for specifying the search space.
from cerebro.tune import hp_choice, hp_uniform, hp_quniform, hp_loguniform, hp_qloguniform

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pyspark.sql import SparkSession
import numpy as np


spark = SparkSession \
    .builder \
    .appName("Cerebro Example") \
    .getOrCreate()

...
work_dir = '/Users/zijian/Desktop/ucsd/cse234/project/cerebro-system/'
backend = SparkBackend(spark_context=spark.sparkContext, num_workers=1)
store = LocalStore(prefix_path=work_dir + 'test/')

df = spark.read.format("libsvm") \
    .option("numFeatures", "784") \
    .load("/Users/zijian/Desktop/ucsd/cse234/project/mnist/mnist.scale")

21/12/01 20:07:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/01 20:07:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


CEREBRO => Time: 2021-12-01 20:07:34, Running 1 Workers


In [2]:
df.count()

60000

In [3]:
from pyspark.ml.feature import OneHotEncoderEstimator

encoder = OneHotEncoderEstimator(dropLast=False)
encoder.setInputCols(["label"])
encoder.setOutputCols(["label_OHE"])

encoder_model = encoder.fit(df)
encoded = encoder_model.transform(df)

feature_columns=['features']
label_columns=['label_OHE']
train_df, test_df = encoded.randomSplit([0.8, 0.2], seed=100)

In [4]:
from keras_tuner.engine import hyperparameters
import autokeras as ak
from cerebro.nas.hphpmodel import HyperHyperModel

img_shape = (28, 28, 1)

input_node = ak.ImageInput()
output_node = ak.ConvBlock(
    kernel_size=hyperparameters.Fixed('kernel_size', value=3),
    num_blocks=hyperparameters.Fixed('num_blocks', value=1),
    num_layers=hyperparameters.Fixed('num_layers', value=2),
)(input_node)
output_node = ak.ClassificationHead()(output_node)
am = HyperHyperModel(input_node, output_node, seed=2000)

am.resource_bind(
    backend=backend, 
    store=store,
    feature_columns=feature_columns,
    label_columns=label_columns,
    evaluation_metric='accuracy', 
)

am.tuner_bind(
    tuner="greedy", 
    hyperparameters=None, 
    objective="val_accuracy",
    max_trials=10,
    overwrite=True,
    exploration=0.3,
)

In [5]:
rel = am.fit(train_df, epochs=3, input_shape=img_shape)

Trial 10 Complete [00h 08m 05s]
val_accuracy: 0.9673958420753479

Best val_accuracy So Far: 0.9765625
Total elapsed time: 00h 55m 32s


In [6]:
rel.metrics

{'model_0_1638418138': {'trial': <keras_tuner.engine.trial.Trial at 0x17d865910>,
  'train_loss': [2.228817156963856, 1.16497028389707, 0.5962928835730287],
  'train_accuracy': [0.311148077249527,
   0.7241836786270142,
   0.8192595839500427],
  'val_loss': [1.9101210645834605, 0.6646093040704727, 0.4766809039687117],
  'val_accuracy': [0.5494791865348816,
   0.8109375238418579,
   0.8585416674613953]},
 'model_1_1638418396': {'trial': <keras_tuner.engine.trial.Trial at 0x17d905090>,
  'train_loss': [2.303350294489234, 2.303333997528089, 2.3033328339978185],
  'train_accuracy': [0.1070091500878334,
   0.10347338020801544,
   0.10664517432451248],
  'val_loss': [2.3032669043540954, 2.3030954241752624, 2.303153946399689],
  'val_accuracy': [0.11708333343267441,
   0.1133333370089531,
   0.11395833641290665]},
 'model_2_1638418815': {'trial': <keras_tuner.engine.trial.Trial at 0x17d9547d0>,
  'train_loss': [0.48381913319201414, 0.09026713565258727, 0.063960221128877],
  'train_accuracy': 

[Stage 11:>                                                         (0 + 1) / 1]